In [1]:
!pip install torch
!pip install torchvision

In [3]:
import torch
import torch.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import unicodedata
import re
import time

print(torch.__version__)

1.7.0


In [5]:
lines = open('./ara.txt', encoding='UTF-8').read().strip().split('\n')

num_examples = 3000
original_word_pairs = [[w for w in l.split('\t')] for l in lines[:num_examples]]

data = pd.DataFrame(original_word_pairs, columns=["eng", "ar","attr"])
data.head(5)

,eng,ar,attr
0,Hi.,مرحبًا.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
1,Run!,اركض!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
2,Help!,النجدة!,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
3,Jump!,اقفز!,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
4,Stop!,قف!,CC-BY 2.0 (France) Attribution: tatoeba.org #4...


In [6]:
import unicodedata as ud
# Converts the unicode file to ascii
# def unicode_to_ascii(s):
#     """
#     Normalizes latin chars with accent to their canonical decomposition
#     """
#     # return ''.join(c for c in unicodedata.normalize('NFD', s)
#     #     if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
    # w = unicode_to_ascii(w.lower().strip())
    
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ." 
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,]+", " ", w)
    
    w = w.rstrip().strip()
    
    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    return w
def arabic_preprocess_sentence(w):
  # retStr = ''
  # for c in w:
  #   if not ud.category(c).startswith('P'):
  #     retStr = retStr.join(c)
  #   else:
  #     retStr = ' ' + retStr.join(c) + ' '
  # return retStr
  w = ''.join(c for c in w if not ud.category(c).startswith('P'))
  return '<start>' + w + '<end>'


In [7]:
# data['ar'].sample(10)

data.ar.apply(lambda w: arabic_preprocess_sentence(w)).sample(10)


2677                    <start>أنت عار<end>
1138             <start>عظيم أليس كذلك<end>
2118            <start>قاد توم السيارة<end>
1970    <start>كنت لا أزال صبيّا آنذاك<end>
610           <start>لم الإستسلام الآن<end>
475         <start>بإمكانه القيام بذلك<end>
243                       <start>ها هي<end>
256                  <start>أنا أعرفها<end>
291                 <start>الحساب عليّ<end>
2648               <start>لم هم غاضبين<end>
Name: ar, dtype: object

In [8]:
# Now we do the preprocessing using pandas and lambdas
data["eng"] = data.eng.apply(lambda w: preprocess_sentence(w))
data["ar"] = data.ar.apply(lambda w: arabic_preprocess_sentence(w))
data.sample(10)

,eng,ar,attr
1734,<start> Turn down the TV . <end>,<start>أخفض صوت التلفاز<end>,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
2875,<start> I m only a customer . <end>,<start>لست إلا زبون<end>,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1368,<start> Turn the TV off . <end>,<start>أطفئ التّلفاز<end>,CC-BY 2.0 (France) Attribution: tatoeba.org #7...
422,<start> Record this . <end>,<start>سجّل هذا<end>,CC-BY 2.0 (France) Attribution: tatoeba.org #8...
2941,<start> She became a doctor . <end>,<start>أصبحت طبيبة<end>,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
690,<start> I need my key . <end>,<start>أحتاج لمفتاحي<end>,CC-BY 2.0 (France) Attribution: tatoeba.org #6...
2626,<start> Where are we going ? <end>,<start>إلى أين نحن ذاهبون<end>,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
87,<start> Let s go ! <end>,<start>هيا بنا<end>,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
902,<start> I have a fever . <end>,<start>لدي حمى<end>,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1625,<start> Listen carefully . <end>,<start>إسمع جيداً<end>,CC-BY 2.0 (France) Attribution: tatoeba.org #2...


In [9]:
# This class creates a word -> index mapping (e.g,. "dad" -> 5) and vice-versa 
# (e.g., 5 -> "dad") for each language,
class LanguageIndex():
    def __init__(self, lang):
        """ lang are the list of phrases from each language"""
        self.lang = lang
        self.word2idx = {}
        self.idx2word = {}
        self.vocab = set()
        
        self.create_index()
        
    def create_index(self):
        for phrase in self.lang:
            # update with individual tokens
            self.vocab.update(phrase.split(' '))
            
        # sort the vocab
        self.vocab = sorted(self.vocab)

        # add a padding token with index 0
        self.word2idx['<pad>'] = 0
        
        # word to index mapping
        for index, word in enumerate(self.vocab):
            self.word2idx[word] = index + 1 # +1 because of pad token
        
        # index to word mapping
        for word, index in self.word2idx.items():
            self.idx2word[index] = word        

In [10]:
# index language using the class above
inp_lang = LanguageIndex(data["ar"].values.tolist())
targ_lang = LanguageIndex(data["eng"].values.tolist())
# Vectorize the input and target languages
input_tensor = [[inp_lang.word2idx[s] for s in ar.split(' ')]  for ar in data["ar"].values.tolist()]
target_tensor = [[targ_lang.word2idx[s] for s in eng.split(' ')]  for eng in data["eng"].values.tolist()]
input_tensor[:10]
target_tensor[:10]

[[5, 120, 3, 4],
 [5, 210, 1, 4],
 [5, 116, 1, 4],
 [5, 136, 1, 4],
 [5, 231, 1, 4],
 [5, 100, 994, 3, 4],
 [5, 100, 994, 3, 4],
 [5, 115, 1, 4],
 [5, 124, 1, 4],
 [5, 124, 1, 4]]

In [11]:
def max_length(tensor):
    return max(len(t) for t in tensor)

In [12]:
# calculate the max_length of input and output tensor
max_length_inp, max_length_tar = max_length(input_tensor), max_length(target_tensor)

In [13]:
def pad_sequences(x, max_len):
    padded = np.zeros((max_len), dtype=np.int64)
    if len(x) > max_len: padded[:] = x[:max_len]
    else: padded[:len(x)] = x
    return padded

In [14]:
# inplace padding
input_tensor = [pad_sequences(x, max_length_inp) for x in input_tensor]
target_tensor = [pad_sequences(x, max_length_tar) for x in target_tensor]
len(target_tensor)

3000

In [15]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val)

(2400, 2400, 600, 600)

In [16]:
from torch.utils.data import Dataset, DataLoader

In [17]:
# conver the data to tensors and pass to the Dataloader 
# to create an batch iterator

class MyData(Dataset):
    def __init__(self, X, y):
        self.data = X
        self.target = y
        # TODO: convert this into torch code is possible
        self.length = [ np.sum(1 - np.equal(x, 0)) for x in X]
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        x_len = self.length[index]
        return x,y,x_len
    
    def __len__(self):
        return len(self.data)

In [18]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
N_BATCH = BUFFER_SIZE//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word2idx)
vocab_tar_size = len(targ_lang.word2idx)

train_dataset = MyData(input_tensor_train, target_tensor_train)
val_dataset = MyData(input_tensor_val, target_tensor_val)

dataset = DataLoader(train_dataset, batch_size = BATCH_SIZE, 
                     drop_last=True,
                     shuffle=True)

In [19]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.gru = nn.GRU(self.embedding_dim, self.enc_units)
        
    def forward(self, x, lens, device):
        # x: batch_size, max_length 
        
        # x: batch_size, max_length, embedding_dim
        x = self.embedding(x) 
                
        # x transformed = max_len X batch_size X embedding_dim
        # x = x.permute(1,0,2)
        x = pack_padded_sequence(x, lens) # unpad
    
        self.hidden = self.initialize_hidden_state(device)
        
        # output: max_length, batch_size, enc_units
        # self.hidden: 1, batch_size, enc_units
        output, self.hidden = self.gru(x, self.hidden) # gru returns hidden state of all timesteps as well as hidden state at last timestep
        
        # pad the sequence to the max length in the batch
        output, _ = pad_packed_sequence(output)
        
        return output, self.hidden

    def initialize_hidden_state(self, device):
        return torch.zeros((1, self.batch_sz, self.enc_units)).to(device)

In [20]:
### sort batch function to be able to use with pad_packed_sequence
def sort_batch(X, y, lengths):
    lengths, indx = lengths.sort(dim=0, descending=True)
    X = X[indx]
    y = y[indx]
    return X.transpose(0,1), y, lengths # transpose (batch x seq) to (seq x batch)

In [21]:
### Testing Encoder part
# TODO: put whether GPU is available or not
# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

encoder.to(device)
# obtain one sample from the data iterator
it = iter(dataset)
x, y, x_len = next(it)

# sort the batch first to be able to use with pac_pack_sequence
xs, ys, lens = sort_batch(x, y, x_len)

enc_output, enc_hidden = encoder(xs.to(device), lens, device)

print(enc_output.size()) # max_length, batch_size, enc_units

torch.Size([5, 64, 1024])


In [22]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, dec_units, enc_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.enc_units = enc_units
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.gru = nn.GRU(self.embedding_dim + self.enc_units, 
                          self.dec_units,
                          batch_first=True)
        self.fc = nn.Linear(self.enc_units, self.vocab_size)
        
        # used for attention
        self.W1 = nn.Linear(self.enc_units, self.dec_units)
        self.W2 = nn.Linear(self.enc_units, self.dec_units)
        self.V = nn.Linear(self.enc_units, 1)
    
    def forward(self, x, hidden, enc_output):
        # enc_output original: (max_length, batch_size, enc_units)
        # enc_output converted == (batch_size, max_length, hidden_size)
        enc_output = enc_output.permute(1,0,2)
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        hidden_with_time_axis = hidden.permute(1, 0, 2)
        
        # score: (batch_size, max_length, hidden_size) # Bahdanaus's
        # we get 1 at the last axis because we are applying tanh(FC(EO) + FC(H)) to self.V
        # It doesn't matter which FC we pick for each of the inputs
        score = torch.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis))
        
        #score = torch.tanh(self.W2(hidden_with_time_axis) + self.W1(enc_output))
          
        # attention_weights shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        attention_weights = torch.softmax(self.V(score), dim=1)
        
        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * enc_output
        context_vector = torch.sum(context_vector, dim=1)
        
        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        # takes case of the right portion of the model above (illustrated in red)
        x = self.embedding(x)
        
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        #x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        # ? Looks like attention vector in diagram of source
        x = torch.cat((context_vector.unsqueeze(1), x), -1)
        
        # passing the concatenated vector to the GRU
        # output: (batch_size, 1, hidden_size)
        output, state = self.gru(x)
        
        
        # output shape == (batch_size * 1, hidden_size)
        output =  output.view(-1, output.size(2))
        
        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)
        
        return x, state, attention_weights
    
    def initialize_hidden_state(self):
        return torch.zeros((1, self.batch_sz, self.dec_units))

In [23]:
# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

encoder.to(device)
# obtain one sample from the data iterator
it = iter(dataset)
x, y, x_len = next(it)

print("Input: ", x.shape)
print("Output: ", y.shape)

# sort the batch first to be able to use with pac_pack_sequence
xs, ys, lens = sort_batch(x, y, x_len)

enc_output, enc_hidden = encoder(xs.to(device), lens, device)
print("Encoder Output: ", enc_output.shape) # batch_size X max_length X enc_units
print("Encoder Hidden: ", enc_hidden.shape) # batch_size X enc_units (corresponds to the last state)

decoder = Decoder(vocab_tar_size, embedding_dim, units, units, BATCH_SIZE)
decoder = decoder.to(device)

#print(enc_hidden.squeeze(0).shape)

dec_hidden = enc_hidden#.squeeze(0)
dec_input = torch.tensor([[targ_lang.word2idx['<start>']]] * BATCH_SIZE)
print("Decoder Input: ", dec_input.shape)
print("--------")

for t in range(1, y.size(1)):
    # enc_hidden: 1, batch_size, enc_units
    # output: max_length, batch_size, enc_units
    predictions, dec_hidden, _ = decoder(dec_input.to(device), 
                                         dec_hidden.to(device), 
                                         enc_output.to(device))
    
    print("Prediction: ", predictions.shape)
    print("Decoder Hidden: ", dec_hidden.shape)
    
    #loss += loss_function(y[:, t].to(device), predictions.to(device))
    
    dec_input = y[:, t].unsqueeze(1)
    print(dec_input.shape)
    break

Input:  torch.Size([64, 8])
Output:  torch.Size([64, 9])
Encoder Output:  torch.Size([5, 64, 1024])
Encoder Hidden:  torch.Size([1, 64, 1024])
Decoder Input:  torch.Size([64, 1])
--------
Prediction:  torch.Size([64, 1474])
Decoder Hidden:  torch.Size([1, 64, 1024])
torch.Size([64, 1])


In [27]:
criterion = nn.CrossEntropyLoss()

def loss_function(real, pred):
    """ Only consider non-zero inputs in the loss; mask needed """
    #mask = 1 - np.equal(real, 0) # assign 0 to all above 0 and 1 to all 0s
    #print(mask)
    mask = real.ge(1).type(torch.FloatTensor)
    
    loss_ = criterion(pred, real) * mask 
    return torch.mean(loss_)

In [28]:
# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## TODO: Combine the encoder and decoder into one class
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, units, BATCH_SIZE)

encoder.to(device)
decoder.to(device)

optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), 
                       lr=0.001)

In [29]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()
    
    encoder.train()
    decoder.train()
    
    total_loss = 0
    
    for (batch, (inp, targ, inp_len)) in enumerate(dataset):
        loss = 0
        
        xs, ys, lens = sort_batch(inp, targ, inp_len)
        enc_output, enc_hidden = encoder(xs.to(device), lens, device)
        dec_hidden = enc_hidden
        
        # use teacher forcing - feeding the target as the next input (via dec_input)
        dec_input = torch.tensor([[targ_lang.word2idx['<start>']]] * BATCH_SIZE)
        
        # run code below for every timestep in the ys batch
        for t in range(1, ys.size(1)):
            predictions, dec_hidden, _ = decoder(dec_input.to(device), 
                                         dec_hidden.to(device), 
                                         enc_output.to(device))
            loss += loss_function(ys[:, t].to(device), predictions.to(device))
            #loss += loss_
            dec_input = ys[:, t].unsqueeze(1)
            
        
        batch_loss = (loss / int(ys.size(1)))
        total_loss += batch_loss
        
        optimizer.zero_grad()
        
        loss.backward()

        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.detach().item()))
        
        
    ### TODO: Save checkpoint for model
    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / N_BATCH))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))
    #Saving a checkpoint
    torch.save(checkpoint, 'checkpoint.pth')
    
    
    
    

Epoch 1 Batch 0 Loss 4.6100
Epoch 1 Loss 2.7914
Time taken for 1 epoch 38.19113278388977 sec

Epoch 2 Batch 0 Loss 2.1263
Epoch 2 Loss 1.8545
Time taken for 1 epoch 42.49919104576111 sec

Epoch 3 Batch 0 Loss 1.4123
Epoch 3 Loss 1.4357
Time taken for 1 epoch 47.780601978302 sec

Epoch 4 Batch 0 Loss 1.1534
Epoch 4 Loss 1.0765
Time taken for 1 epoch 51.15610718727112 sec

Epoch 5 Batch 0 Loss 0.7623
Epoch 5 Loss 0.7651
Time taken for 1 epoch 51.79044008255005 sec

Epoch 6 Batch 0 Loss 0.5025
Epoch 6 Loss 0.5073
Time taken for 1 epoch 52.67750811576843 sec

Epoch 7 Batch 0 Loss 0.3992
Epoch 7 Loss 0.3143
Time taken for 1 epoch 50.2421498298645 sec

Epoch 8 Batch 0 Loss 0.2258
Epoch 8 Loss 0.1938
Time taken for 1 epoch 52.22065305709839 sec

Epoch 9 Batch 0 Loss 0.0718
Epoch 9 Loss 0.1299
Time taken for 1 epoch 52.53693699836731 sec

Epoch 10 Batch 0 Loss 0.0756
Epoch 10 Loss 0.0975
Time taken for 1 epoch 49.83271789550781 sec

